## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
model_path_list = glob.glob('/kaggle/input/bert-131-best/' + '*.h5')
model_path_list.sort()
print('Models to predict:', model_path_list)

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict: ['/kaggle/input/bert-131-best/131-Bert_base_unc Quest_Ans3_model_fold_1.h5', '/kaggle/input/bert-131-best/131-Bert_base_unc Quest_Ans3_model_fold_2.h5', '/kaggle/input/bert-131-best/131-Bert_base_unc Quest_Ans3_model_fold_3.h5', '/kaggle/input/bert-131-best/131-Bert_base_unc Quest_Ans3_model_fold_4.h5', '/kaggle/input/bert-131-best/131-Bert_base_unc Quest_Ans3_model_fold_5.h5']
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
text_features_question = ['question_title', 'question_body']
text_features_answer = ['question_title', 'answer']

# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
N_CLASS_QUESTION = len(question_target_cols)
N_CLASS_ANSWER = len(answer_target_cols)
MAX_SEQUENCE_LENGTH = 512
MAX_LENGTH_TITLE = 30
MAX_LENGTH_QUESTION = 479
MAX_LENGTH_ANSWER = 479

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays_2(test, text_features_question, tokenizer, MAX_SEQUENCE_LENGTH, 
                               MAX_LENGTH_TITLE, MAX_LENGTH_QUESTION, ignore_first_sep=False)\
        + compute_input_arays_2(test, text_features_answer, tokenizer, MAX_SEQUENCE_LENGTH, 
                                MAX_LENGTH_TITLE, MAX_LENGTH_ANSWER, ignore_first_sep=False)

# Model

In [7]:
def model_fn():
    input_word_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_question')
    input_masks_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_question')
    segment_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_question')

    input_word_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_answer')
    input_masks_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_answer')
    segment_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_answer')

    bert_layer = KerasLayer(BERT_PATH, trainable=True)
    pooled_output_question, sequence_output_question = bert_layer([input_word_ids_question, input_masks_question, segment_ids_question])
    pooled_output_answer, sequence_output_answer = bert_layer([input_word_ids_answer, input_masks_answer, segment_ids_answer])

    x_question = GlobalAveragePooling1D()(sequence_output_question)
    x_question = Dropout(0.2)(x_question)
    output_question = Dense(N_CLASS_QUESTION, kernel_initializer='glorot_uniform', activation="sigmoid", name="outputquestion")(x_question)
    
    x_answer = GlobalAveragePooling1D()(sequence_output_answer)
    x_answer = Dropout(0.2)(x_answer)
    output_answer = Dense(N_CLASS_ANSWER, kernel_initializer='glorot_uniform', activation="sigmoid", name="output_answer")(x_answer)
    
    output = Concatenate()([output_question, output_answer])

    model = Model(inputs=[input_word_ids_question, input_masks_question, segment_ids_question, 
                          input_word_ids_answer, input_masks_answer, segment_ids_answer], outputs=output)

    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))
weights = [.25, .2, .1, .2, .25]

for index, model_path in enumerate(model_path_list):
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) * weights[index]

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.943935,0.712709,0.157413,0.553382,0.663123,0.578083,0.665551,0.589613,0.556513,...,0.909709,0.932269,0.569184,0.970946,0.973962,0.823893,0.038824,0.036383,0.765140,0.919211
1,46,0.825712,0.494429,0.012105,0.745578,0.762355,0.926517,0.520372,0.439363,0.105881,...,0.658154,0.925986,0.646692,0.963313,0.978078,0.864696,0.886150,0.150880,0.115755,0.882394
2,70,0.865823,0.700038,0.037951,0.762179,0.831348,0.887805,0.585423,0.498790,0.123376,...,0.859655,0.937837,0.562032,0.967326,0.962738,0.826061,0.046115,0.036941,0.812085,0.911992
3,132,0.842071,0.415406,0.013157,0.719432,0.677146,0.899617,0.511090,0.393147,0.110690,...,0.688391,0.945945,0.687456,0.972472,0.978561,0.898330,0.747460,0.163954,0.570931,0.895587
4,200,0.918375,0.504052,0.043185,0.825558,0.762781,0.880317,0.643175,0.567461,0.103240,...,0.686776,0.919688,0.649809,0.956731,0.962651,0.814702,0.328880,0.124445,0.593149,0.890043


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.876731,0.590298,0.036955,0.722062,0.781065,0.856582,0.564483,0.475884,0.223753,...,0.786532,0.931418,0.659501,0.962869,0.972354,0.865697,0.507699,0.128088,0.507966,0.906339
std,2812.670060,0.048670,0.141322,0.049907,0.091085,0.092644,0.108351,0.056099,0.088989,0.185945,...,0.089991,0.025151,0.047914,0.012517,0.011247,0.039141,0.297124,0.055377,0.260149,0.022174
min,39.000000,0.735146,0.357549,0.004584,0.336537,0.451709,0.334100,0.453717,0.341624,0.017777,...,0.574715,0.795570,0.489279,0.912113,0.905494,0.739103,0.016419,0.014056,0.043581,0.831638
25%,2572.000000,0.843474,0.470514,0.011906,0.677436,0.728069,0.839022,0.518196,0.405570,0.083081,...,0.708660,0.918134,0.627465,0.956849,0.967107,0.842454,0.212460,0.086379,0.291103,0.892013
50%,5093.000000,0.878999,0.576051,0.017515,0.724769,0.788421,0.894775,0.554948,0.446610,0.148923,...,0.784815,0.936475,0.654931,0.964701,0.975292,0.870403,0.567223,0.131949,0.502388,0.908973
75%,7482.000000,0.915805,0.714498,0.036531,0.776380,0.841277,0.921177,0.602544,0.529463,0.331362,...,0.873681,0.949216,0.687898,0.971101,0.980115,0.895275,0.782568,0.167936,0.731760,0.922686
max,9640.000000,0.968128,0.901339,0.371083,0.962035,0.966657,0.964588,0.734563,0.724058,0.806702,...,0.950336,0.980595,0.830115,0.985788,0.989771,0.945728,0.931924,0.270478,0.961298,0.950564
